## 1. Изучение данных

Загрузим необходимые библиотеки и функции.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Загрузим файл и изучим данные.

In [2]:
mobile = pd.read_csv('/datasets/users_behavior.csv')
mobile.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [3]:
mobile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


## 2. Разбиение данных на выборки

Разобъем данные на выборки: сначала на обучающую и валидационную+проверочную в соотношении 60:40, а затем меньшую еще раз на собственно валидационную и проверочную в соотношении 50:50. Получилось 3 выборки 60:20:20.

In [4]:
mobile_train, mobile_valid_test = train_test_split(mobile, test_size=0.4, random_state=12345)
mobile_valid, mobile_test = train_test_split(mobile_valid_test, test_size=0.5, random_state=12345)

Объявим признаки и целевые признаки для каждой выборки.

In [5]:
features_train = mobile_train.drop('is_ultra', axis=1)
target_train = mobile_train['is_ultra']
features_valid = mobile_valid.drop('is_ultra', axis=1)
target_valid = mobile_valid['is_ultra']
features_test = mobile_test.drop('is_ultra', axis=1)
target_test = mobile_test['is_ultra']

## 3. Исследование моделей

Предсказывать необходимо категорию - какой из двух тарифов выбрать. Поэтому в качестве моделей выберем решающее дерево, случайный лес и логистическую регрессию.  
Для дерева в цикле будем менять гиперпараметры max_depth, splitter и criterion.

<font color=blue>Стоит задача классификации, поэтому выбераем классификаторы.

In [6]:
acc = 0
crit_list = ['gini', 'entropy']
split_list = ['best', 'random']
for crit in crit_list:
    for split in split_list:
        for depth in range(1, 21, 1):
            three_model = DecisionTreeClassifier(criterion=crit, splitter=split, random_state=12345, max_depth=depth)
            three_model.fit(features_train, target_train)
            prediction = three_model.predict(features_valid)
            accuracy = accuracy_score(target_valid, prediction)
            if accuracy > acc:
                acc = accuracy
                dep = depth
                cr = crit
                spl = split
print('Наибольшее значение accuracy -', acc, 'при:')
print('max_depth =', dep)
print('splitter =', spl)
print('criterion =', cr)

Наибольшее значение accuracy - 0.7884914463452566 при:
max_depth = 7
splitter = random
criterion = entropy


Для случайного леса меняем в цикле гиперпараметры min_samples_split, max_depth и n_estimators.

In [7]:
acc = 0
for mss in range(2,5,1):
    for depth in range(1, 11, 1):
        for estim in range(10, 101, 10):
            forest_model = RandomForestClassifier(random_state=12345, n_estimators=estim, max_depth=depth, min_samples_split=mss)
            forest_model.fit(features_train, target_train)
            prediction = forest_model.predict(features_valid)
            accuracy = accuracy_score(target_valid, prediction)
            if accuracy > acc:
                acc = accuracy
                est = estim
                dep = depth
                ms = mss
print('Наибольшее значение accuracy -', acc, 'при:')
print('max_depth =', dep)
print('n_estimators =', est)
print('min_samples_split =', ms)

Наибольшее значение accuracy - 0.8102643856920684 при:
max_depth = 7
n_estimators = 50
min_samples_split = 3


Для логистической регрессии установим параметр solver='liblinear', что является хорошим выбором для небольших датасетов.

In [8]:
regress_model = LogisticRegression(random_state=12345, solver='liblinear')
regress_model.fit(features_train, target_train)
regress_model.predict(features_valid)
accuracy = regress_model.score(features_valid, target_valid)
print('Значение accuracy -', accuracy)

Значение accuracy - 0.7589424572317263


Итак, победу одержала модель "случайный лес" со значением n_estimators = 50. Не сильно отстало и решающее дерево, при том, что оно проще и считается намного быстрее (впрочем, на маленьких выборках это не принципиально). На последнем месте с небольшим отрывом - логистическая регрессия.

## 4. Проверка модели на тестовой выборке

Проверим лучшую модель на тестовой выборке. Но сначала объединим для обучения обучающую и валидационную выборки. Тестовых данных в ней не будет, так что все честно.

In [9]:
model = RandomForestClassifier(random_state=12345, max_depth=7, n_estimators=50, min_samples_split=3)

mob = pd.concat([mobile_train, mobile_valid])
features = mob.drop('is_ultra', axis=1)
target = mob['is_ultra']
model.fit(features, target)

prediction = model.predict(features_test)
accuracy = accuracy_score(target_test, prediction)
print(accuracy)

0.7962674961119751


In [10]:
model_old = RandomForestClassifier(random_state=12345, n_estimators=50)

model_old.fit(features, target)

prediction_old = model_old.predict(features_test)
accuracy_old = accuracy_score(target_test, prediction_old)
print(accuracy_old)

0.80248833592535


Как видим, значение accuracy на тестовой выборке немного уменьшилось по сравнению с валидационной, а также по сравнению с моделью без "подкрутки" настроек (хотя мы взяли гиперпараметры от модели, показавшей лучшие результаты на валидационной выборке). Похоже на переобучение.

## 5. Проверка модели на адекватность

Проверим, какое значение accuracy получится, если мы не будем заморачиваться и во все ответы подставим нули.

In [11]:
1 - mobile['is_ultra'].mean()

0.693528313627878

Доля нулей - 69%, в данном случае это и будет точность предсказания. Наша же модель предсказывает с точностью 80%, что более адекватно.